# PuLP で輸送問題 (Transportation Problem) を解く  

PuLP は Python で線形計画 (Linear Programing) 問題をモデル化し、ソルバーで解くためのツールです。  

- [coin-or/pulp: A python Linear Programming API](https://github.com/coin-or/pulp)  
- [Optimization with PuLP](https://coin-or.github.io/pulp/)  
- [pulp: Pulp classes](https://coin-or.github.io/pulp/technical/pulp.html)  

Anaconda Cloud ではパッケージは公開されていませんので、`pip install pulp` でインストールしてください。  

## 問題  

あるビール醸造所はふたつの倉庫を保有しており、そこから 5 つのバーにビールを供給しています。  
毎週始めにそれぞれのバーは醸造所の事務所に大量のビールの注文を送り、それからビールは適切な倉庫から発送されます。  
醸造所はコンピュータプログラムを使ってどの倉庫からどのバーにビールを供給すればコストを最小化できるかを計算しています。  
例えば、

- A 倉庫には 1000 ケース、B 倉庫には 4000 ケース の在庫がある  
- それぞれのバーは 500 ケース、900 ケース、1800 ケース、200 ケース、700 ケース  

|From Warehouse to Bar|A|B|
|--- |--- |--- |
|1|2|3|
|2|4|1|
|3|5|3|
|4|2|2|
|5|1|3|



$$\begin{split}F &= \{A,B,C\} \\
O &= \{1, 2, 3, 4, 5 \} \\
x_{(f,o)} &\ge 0 \ldots \forall f \in F, o \in O \\
x_{(f,o)} &\in \mathbb{Z}^+ \ldots \forall f \in F, o \in O \\\end{split}$$

In [1]:
# Import PuLP modeller functions
from pulp import *

In [2]:
# Creates a list of all the supply nodes
factories = ["A", "B", "C"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 120,
          "B": 125,
          "C": 75}

# Creates a list of all demand nodes
offices = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"1": 100,
          "2": 60,
          "3": 40,
          "4": 75,
          "5": 25}

In [3]:
# Creates a list of costs of each transportation path
costs = [# offices
        # 1  2  3  4  5
         [4, 3, 4, 5, 2], # A  factories
         [5, 2, 3, 5, 3], # B
         [2, 1, 6, 4, 3]  # C
        ]

In [4]:
# Creates the prob variable to contain the problem data
prob = LpProblem("Beer_Distribution_Problem", LpMinimize)

In [5]:
# Creates a list of tuples containing all the possible routes for transport
Routes = [(f, o) for f in factories for o in offices]

In [6]:
# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = LpVariable.dicts("Route", (factories, offices), 0, None, LpInteger)

In [7]:
# The objective function is added to prob first
prob += lpSum([route_vars[f][o]*costs[factories.index(f)][int(o)-1] for (f, o) in Routes]), "Sum of Transporting Costs"

In [8]:
# The supply maximum constraints are added to prob for each supply node (warehouse)
for f in factories:
    prob += lpSum([route_vars[f][o] for o in offices]) <= supply[f], "Sum of Products out of Warehouse %s"%f

# The demand minimum constraints are added to prob for each demand node (bar)
for o in offices:
    prob += lpSum([route_vars[f][o] for f in factories]) >= demand[o], "Sum of Products into Bars %s"%o

In [9]:
prob.solve()

1

In [10]:
# 結果の表示
for v in prob.variables():
    print(v.name, ":", pulp.value(v))
print("総コスト:", pulp.value(prob.objective))

Route_A_1 : 25.0
Route_A_2 : 0.0
Route_A_3 : 0.0
Route_A_4 : 70.0
Route_A_5 : 25.0
Route_B_1 : 0.0
Route_B_2 : 60.0
Route_B_3 : 40.0
Route_B_4 : 5.0
Route_B_5 : 0.0
Route_C_1 : 75.0
Route_C_2 : 0.0
Route_C_3 : 0.0
Route_C_4 : 0.0
Route_C_5 : 0.0
総コスト: 915.0


In [ ]:
# Creates a list of all the supply nodes
factories = ["A", "B", "C"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 120,
          "B": 125,
          "C": 75}

# Creates a list of all demand nodes
offices = ["1", "2", "3", "4", "5", "d"]  # d for "Dummy"

# Creates a dictionary for the number of units of demand for each demand node
demand = {"1": 95,
          "2": 50,
          "3": 40,
          "4": 50,
          "5": 15,
          "d": 50}

## 参考文献  

- [A Transportation Problem — PuLP 2.0 documentation](https://coin-or.github.io/pulp/CaseStudies/a_transportation_problem.html)  
- [問題解決のためのオペレーションズ・リサーチ入門｜日本評論社](https://www.nippyo.co.jp/shop/book/1404.html)  